In [1]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score

import keras
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import RMSprop, Adam, SGD

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

from os.path import join, basename, exists
from os import makedirs, listdir

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preproc_target_train(data, reduce_classes=False):
    diagnoses = data['Код_диагноза'].copy()
    
    if reduce_classes:
        pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    else:
        pop_diagnoses = set(diagnoses)
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

In [3]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)

In [12]:
class NNModel(BaseEstimator, TransformerMixin):
    def __init__(self, batch_size, epochs, tb_log_dir, model_path, validation_split=0.3):
        self.batch_size = batch_size
        self.epochs = epochs
        self.tb_log_dir = tb_log_dir
        self.model_path = model_path
        self.validation_split = validation_split
    
    def fit(self, x, y=None):
        self.classes = np.unique(y)
        self.classes_voc = dict(zip(self.classes, range(self.classes.shape[0])))
        self.voc_classes = dict(zip(range(self.classes.shape[0]), self.classes))
        y_proc = np.zeros((y.shape[0], self.classes.shape[0]), dtype=np.float32)
        for i, yc in enumerate(y):
            y_proc[i, self.classes_voc[yc]] = 1.
        
        self.model = Sequential([
            InputLayer(input_shape=(x.shape[1],)),
            Dense(4096, activation='sigmoid'),
            Dense(self.classes.shape[0], activation='softmax')
        ])
        
        optim = RMSprop(lr=1e-4, decay=1e-6)
        self.model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.fit(x, y_proc,
                  batch_size=self.batch_size, epochs=self.epochs,
                  callbacks=[
                      TensorBoard(log_dir=self.tb_log_dir, batch_size=self.batch_size),
                      ModelCheckpoint(filepath=self.model_path, monitor='acc', period=5)
                  ],
                  validation_split=self.validation_split)
        
        return self
        
    def transform(self, x):
        pred = self.model.predict(x)
        max_idxs = np.argmax(pred, axis=1)
        
        return np.array(list(map(lambda max_idx: self.voc_classes[max_idx], max_idxs)))

In [13]:
experiment_dir = 'simple_models'
model_name = 'nn' + str(utils.get_next_model_id(experiment_dir))
tb_log_dir = join(experiment_dir, 'log', model_name)
models_dir = join(experiment_dir, 'models')
model_path = utils.get_model_fname_pattern(models_dir, model_name)

pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('complaints_pipe', Pipeline([
                ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=10, stop_words=stopwords.words('russian')))
            ])),
            ('complaints_n_pipe', Pipeline([
                ('complaint_n_selector', ItemSelector(key='Жалобы (ngramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
            ])),
#             ('doctor_pipe', Pipeline([
#                 ('doctor_selector', ItemSelector(key='Врач')),
#                 ('doc_pop', DoctorsPopularityTransformator()),
#                 ('count_vect', CountVectorizer())
#             ])),
            ('gender_pipe', Pipeline([
                ('gender_selector', ItemSelector(key='Пол')),
                ('gender_transform', GenderTransformator())
            ])),
            ('age_pipe', Pipeline([
                ('age_selector', ItemSelector(key='Возраст')),
                ('age_transformator', AgeTransformator())
            ]))
        ]
    )),
    ('clf', NNModel(batch_size=128, epochs=100, tb_log_dir=tb_log_dir,
                    model_path=model_path, validation_split=0.3))
])

In [14]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
# train, valid = train_test_split(train, test_size=0.3)

In [15]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)
# valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [16]:
np.unique(train_y).shape

(2302,)

In [ ]:
pipe.fit(train, train_y)

Train on 43383 samples, validate on 18593 samples
Epoch 1/100
43383/43383 [==============================] - 10s 227us/step - loss: 5.7470 - acc: 0.0705 - val_loss: 6.6203 - val_acc: 0.0830
Epoch 2/100
43383/43383 [==============================] - 9s 219us/step - loss: 5.5697 - acc: 0.1133 - val_loss: 6.5280 - val_acc: 0.1198
Epoch 3/100
43383/43383 [==============================] - 10s 236us/step - loss: 5.4074 - acc: 0.1464 - val_loss: 6.3704 - val_acc: 0.1404
Epoch 4/100
43383/43383 [==============================] - 10s 223us/step - loss: 5.2254 - acc: 0.1774 - val_loss: 6.2335 - val_acc: 0.1585
Epoch 5/100
43383/43383 [==============================] - 10s 226us/step - loss: 5.0578 - acc: 0.2015 - val_loss: 6.0922 - val_acc: 0.1748
Epoch 6/100
43383/43383 [==============================] - 10s 220us/step - loss: 4.8932 - acc: 0.2214 - val_loss: 5.9440 - val_acc: 0.1991
Epoch 7/100
43383/43383 [==============================] - 10s 222us/step - loss: 4.7370 - acc: 0.2322 - val_lo

Epoch 59/100
43383/43383 [==============================] - 10s 220us/step - loss: 3.1108 - acc: 0.3677 - val_loss: 4.5739 - val_acc: 0.2892
Epoch 60/100
43383/43383 [==============================] - 10s 237us/step - loss: 3.1060 - acc: 0.3697 - val_loss: 4.5757 - val_acc: 0.2860
Epoch 61/100
43383/43383 [==============================] - 10s 225us/step - loss: 3.1015 - acc: 0.3713 - val_loss: 4.5740 - val_acc: 0.2863
Epoch 62/100
43383/43383 [==============================] - 10s 220us/step - loss: 3.1004 - acc: 0.3709 - val_loss: 4.5529 - val_acc: 0.2889
Epoch 63/100
43383/43383 [==============================] - 10s 231us/step - loss: 3.0840 - acc: 0.3729 - val_loss: 4.5551 - val_acc: 0.2921
Epoch 64/100
43383/43383 [==============================] - 10s 224us/step - loss: 3.0821 - acc: 0.3721 - val_loss: 4.5493 - val_acc: 0.2885
Epoch 65/100
43383/43383 [==============================] - 10s 220us/step - loss: 3.0737 - acc: 0.3741 - val_loss: 4.5515 - val_acc: 0.2906
Epoch 66/100


In [23]:
pred = pipe.transform(train)

In [95]:
np.unique(pred, return_counts=True)

(array(['A46', 'A49.3', 'A63.0', 'A63.8', 'A66.1', 'B00', 'B00.1', 'B00.8',
        'B00.9', 'B01.9', 'B02', 'B02.9', 'B07', 'B08.1', 'B35.1', 'B35.3',
        'B35.4', 'B35.6', 'B36.0', 'B37', 'B37.2', 'B37.3+', 'B37.4+',
        'B97.1', 'B97.7', 'C20', 'C44.5', 'C61', 'D17.0', 'D17.1', 'D17.2',
        'D18.0', 'D21', 'D22.5', 'D23', 'D23.3', 'D23.4', 'D23.5', 'D23.6',
        'D23.7', 'D23.9', 'D24', 'D25', 'D32.1', 'D35.2', 'D50', 'D50.0',
        'D50.9', 'D64.9', 'D72.1', 'D80.2', 'D89.8', 'E01.0', 'E01.1',
        'E01.8', 'E03.8', 'E03.9', 'E04.1', 'E04.2', 'E05', 'E05.0',
        'E05.8', 'E05.9', 'E06.1', 'E06.3', 'E10', 'E11', 'E13', 'E14',
        'E22.1', 'E28', 'E28.1', 'E28.8', 'E28.9', 'E29.1', 'E34', 'E55',
        'E66', 'E66.0', 'E78.0', 'E89.0', 'F34.1', 'F40.0', 'F41.0',
        'F41.1', 'F41.2', 'F41.9', 'F43.2', 'F45', 'F45.3', 'F45.9',
        'F52.2', 'F95.0', 'G24', 'G24.9', 'G43.0', 'G43.1', 'G44.1',
        'G44.2', 'G45.0', 'G50', 'G50.0', 'G50.8', 'G51', 

## Submit

In [27]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [28]:
pipe.fit(train, train_y)

Epoch 1/100
61976/61976 [==============================] - 6s 94us/step - loss: 5.8942 - acc: 0.0683
Epoch 2/100
61976/61976 [==============================] - 6s 93us/step - loss: 5.6908 - acc: 0.1080
Epoch 3/100
61976/61976 [==============================] - 6s 93us/step - loss: 5.5446 - acc: 0.1354
Epoch 4/100
61976/61976 [==============================] - 6s 93us/step - loss: 5.3526 - acc: 0.1662
Epoch 5/100
61976/61976 [==============================] - 6s 94us/step - loss: 5.1235 - acc: 0.1936
Epoch 6/100
61976/61976 [==============================] - 6s 91us/step - loss: 4.8901 - acc: 0.2130
Epoch 7/100
61976/61976 [==============================] - 6s 93us/step - loss: 4.6735 - acc: 0.2288
Epoch 8/100
61976/61976 [==============================] - 6s 93us/step - loss: 4.4935 - acc: 0.2394
Epoch 9/100
61976/61976 [==============================] - 6s 95us/step - loss: 4.3432 - acc: 0.2480
Epoch 10/100
61976/61976 [==============================] - 6s 93us/step - loss: 4.2246 - a

61976/61976 [==============================] - 6s 95us/step - loss: 3.1874 - acc: 0.3607
Epoch 82/100
61976/61976 [==============================] - 6s 94us/step - loss: 3.1842 - acc: 0.3620
Epoch 83/100
61976/61976 [==============================] - 6s 93us/step - loss: 3.1804 - acc: 0.3622
Epoch 84/100
61976/61976 [==============================] - 6s 93us/step - loss: 3.1768 - acc: 0.3624
Epoch 85/100
61976/61976 [==============================] - 6s 95us/step - loss: 3.1741 - acc: 0.3627
Epoch 86/100
61976/61976 [==============================] - 6s 94us/step - loss: 3.1702 - acc: 0.3621
Epoch 87/100
61976/61976 [==============================] - 6s 94us/step - loss: 3.1687 - acc: 0.3638
Epoch 88/100
61976/61976 [==============================] - 6s 93us/step - loss: 3.1635 - acc: 0.3642
Epoch 89/100
61976/61976 [==============================] - 6s 93us/step - loss: 3.1593 - acc: 0.3647
Epoch 90/100
61976/61976 [==============================] - 6s 94us/step - loss: 3.1541 - acc: 

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e... NNModel(batch_size=128, epochs=100, tb_log_dir='simple_models/log/nn2',
    validation_split=0.0))])

In [30]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')

In [32]:
test_pred = pipe.transform(test)

In [33]:
submit = pd.DataFrame({'Id_Записи': test['Id_Записи'], 'Код_диагноза': test_pred})

In [34]:
submit['Код_диагноза'].value_counts()

M42.1     2960
Z00.0     2517
Z32.1     1761
N76.0     1497
J06.9     1226
J00       1133
K29.9     1130
M65       1006
N77.1*     697
I11        591
I83.9      556
K30        534
D23.9      513
Z01.8      506
J35.0      473
N60.1      461
N41.1      387
Z01.4      386
D25        385
H52.1      371
G90        355
N95.2      339
H35.0      310
M54.8      300
K29.5      269
M54.5      264
M54.6      260
M41        256
B07        235
N30.2      234
          ... 
L20.8        3
I84.4        3
I20.9        3
N47          3
H00.0        3
N80.0        3
N34.1        3
B02          2
D18.0        2
A63.0        2
L28.0        2
L71          2
L23          1
M42.0        1
J01.1        1
S90.1        1
L30.9        1
M12.5        1
J45.8        1
G93.4        1
L08.8        1
N30          1
N11          1
K04.0        1
H68.1        1
H90.3        1
M45          1
M79.2        1
M70.2        1
H01.0        1
Name: Код_диагноза, Length: 216, dtype: int64

In [35]:
submit.to_csv('submit/bow_simple_nn2_diag_all_compl_uni-n_gram.csv', header=True, index=False)